In [ ]:
def convert_to_coord(coord_str):
    # 分割字符串
    points = coord_str.split(';')

    # 將每個點進一步分割為 x 和 y
    point_coords = [point.split(',') for point in points]

    # 將每對 x 和 y 轉換為浮點數
    float_coords = [(float(x), float(y)) for x, y in point_coords]

    # 將坐標轉換為 x1, y1, x2, y2 的格式
    x1, y1, x2, y2 = float_coords[0][0], float_coords[0][1], float_coords[2][0], float_coords[2][1]

    return x1, y1, x2, y2

In [ ]:
import xml.etree.ElementTree as ET

data_path = "/home/nas/Research_Group/Personal/Ben/ocean_waste_detection/xml2txt/參賽者驗證資料集下載(500)_0912.xml"

tree = ET.parse(data_path)
root = tree.getroot()

results = {}
childs = root.iter()
name = ''
for child in childs:
    if child.tag not in ['image', 'polygon']:
        continue

    if child.tag == 'image':
        name = child.attrib['name']
        results[name] = {"boxes": [], "labels": []}

    elif child.tag == 'polygon':
        label = child.attrib['label']
        points = child.attrib['points']
        points = convert_to_coord(points) # (x1, y1, x2, y2)
        results[name]["boxes"].append(points)
        results[name]["labels"].append(label)

"""
I want to store the results to json file
"""

import json

with open('result.json', 'w', encoding='utf-8') as fp:
    json.dump(results, fp, ensure_ascii=False, indent=3)


In [2]:
"""
I want read the json file
"""
import json
json_path = '/Users/ben/Desktop/Programing/Python/evaluate/result.json'
with open(json_path, 'r', encoding='utf-8') as fp:
    results = json.load(fp)

infos = results['result']

results = {}
for info in infos:
    name = info['IMG_PATH'].split('/')[-1]
    results[name] = {"boxes": [], "labels": [], "scores": []}
    boxes = info['BOUNDING_BOX']

    for box in boxes:
        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
        results[name]["boxes"].append([x1, y1, x2, y2])
        label = box['label_id']
        results[name]["labels"].append(label)
        score = box['score']
        results[name]["scores"].append(score)

with open('pred.json', 'w', encoding='utf-8') as fp:
  json.dump(results, fp, ensure_ascii=False, indent=3)

In [18]:
import json
import numpy as np

def calculate_intersection_area(rect1, rect2):
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]))
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]))

    intersection_area = x_overlap * y_overlap
    return intersection_area

def calculate_iou(box1, box2):
    # 計算兩個框的交集和聯集
    intersection = calculate_intersection_area(box1, box2)
    union = (box1[2] - box1[0]) * (box1[3] - box1[1]) + (box2[2] - box2[0]) * (box2[3] - box2[1]) - intersection

    # 計算 IoU
    iou = intersection / union if union > 0 else 0
    return iou

def calculate_ap(precision, recall):
    # 計算平均精確度 (AP)
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))
    print("mrec", mrec, "mpre", mpre)
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = max(mpre[i - 1], mpre[i])
    i = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def evaluate_mAP(predictions, targets):
    all_boxes = {}
    all_labels = {}
    all_scores = {}

    # 整理預測和真實框的資訊
    for image_id, prediction_info in predictions.items():
        boxes = np.array(prediction_info["boxes"])
        labels = prediction_info["labels"]
        scores = prediction_info["scores"]

        all_boxes[image_id] = boxes
        all_labels[image_id] = labels
        all_scores[image_id] = scores

    # 計算 mAP
    class_ids = [f"WASTE_{i}" for i in range(1, 21)]
    average_precisions = []
    for class_id in class_ids:  # 假設有 20 類別
        class_id = "WASTE_18"
        true_positives = []
        scores = []
        num_annotations = 0

        for image_id, targets_info in targets.items():
            if class_id in targets_info["labels"]:
                num_annotations += 1

            if image_id not in all_boxes:
                continue

            # 選擇預測結果中指定類別的框
            class_mask = np.array(all_labels[image_id]) == class_id
            boxes = []
            box_scores = []
            for i, info in enumerate(class_mask):
                if info:
                    boxes.append(all_boxes[image_id][i])
                    box_scores.append(all_scores[image_id][i])

            if len(boxes) == 0:
                true_positives.append(0)
                scores.append(0)
                continue

            # 選擇真實框中指定類別的框
            true_mask = np.array(targets_info["labels"]) == class_id
            true_boxes = np.array(targets_info["boxes"])[true_mask]

            # 計算 IoU
            ious = np.array([calculate_iou(box, true_box) for box in boxes for true_box in true_boxes])

            # 判斷每個預測框是否為正確檢測
            true_positive = (ious >= 0.5).any()
            true_positives.append(true_positive)
            scores.append(np.max(box_scores))

        print(sum(scores))
        # 計算精確度和召回率
        scores = np.array(scores)
        true_positives = np.array(true_positives)
        indices = np.argsort(-scores)
        true_positives = true_positives[indices]
        cumulative_true_positives = np.cumsum(true_positives)
        recall = cumulative_true_positives / num_annotations
        precision = cumulative_true_positives / np.arange(1, len(true_positives) + 1)
        # 計算 AP 並添加到 average_precisions 中
        ap = calculate_ap(precision, recall)
        if ap:
            average_precisions.append(ap)
        else:
            average_precisions.append(0)
        print(class_id, ap, "Number of objects:", num_annotations)
        break
            

    # 計算 mAP
    mAP = np.mean(average_precisions)
    return mAP

# 使用範例
pred_path = "/Users/ben/Desktop/test.json"
target_path = "/Users/ben/Desktop/result.json"

with open(pred_path, 'r') as f:
    predictions = json.load(f)

with open(target_path, 'r') as f:
    targets = json.load(f)

mAP = evaluate_mAP(predictions, targets)

print(mAP)

16.107234
mrec [ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan

/var/folders/1p/lpdf4tns5dj0rk1jwngwdlbc0000gn/T/ipykernel_82832/896829085.py:95: RuntimeWarning: invalid value encountered in true_divide
  recall = cumulative_true_positives / num_annotations
